In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [361]:
def clean_models(df):
    df = df.drop(['Base model', 'Finetune compute (FLOP)', 'Finetune compute notes'], axis=1)
    df['Domain'] = df['Domain'].astype("string").str.split(",").map(lambda x: sorted(x)).map(lambda x: ','.join(x))
    df['System'] = df['System'].astype("string")
    df['Task'] = df['Task'].astype("string").str.split(',').map(lambda x: sorted(x)).map(lambda x: ','.join(x))
    df['Model accessibility'] = df['Model accessibility'].astype('category')
    df['Citations'] = df['Citations'].fillna(0).astype('uint32')
    df['Publication date'] = pd.to_datetime(df['Publication date'])
    df['Organization'] = df['Organization'].astype('category')
    return df

def clean_bloom(df):
    df = df.drop(['cloud_provider', 'cloud_region'], axis=1)
    df = df[~df.drop(['timestamp', 'project_name', 'run_id'], axis=1).isna().any(axis=1)]
    df.loc[312, 'timestamp'] = '2022-05-18T12:04:08'
    df = df.drop([767])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

models_df = pd.read_csv("large_scale_ai_models_added_cols.csv").drop(['Authors'], axis=1)
bloom_df = pd.read_csv("bloom_emissions.csv")
models_df = clean_models(models_df)
bloom_df = clean_bloom(bloom_df)
models_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 29 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   System                                           223 non-null    string        
 1   Domain                                           223 non-null    object        
 2   Task                                             223 non-null    object        
 3   Model accessibility                              184 non-null    category      
 4   Link                                             220 non-null    object        
 5   Citations                                        223 non-null    uint32        
 6   Reference                                        185 non-null    object        
 7   Publication date                                 223 non-null    datetime64[ns]
 8   Organization                            

In [385]:
models_df['Organization categorization (from Organization)']\
         .astype('string')\
         .str.strip()\
         .str.split()\
         .apply(lambda x: set(x)).value_counts()

Organization categorization (from Organization)
{Industry}                               188
{Industry,Academia}                        9
{Industry,Industry}                        6
{Academia}                                 5
{Government}                               3
{Industry,Industry,Industry}               3
{Academia,Industry}                        3
{collective,Academia, Research}            2
{Academia,Industry,Academia,Academia}      1
{collective, Industry,Research}            1
{Academia,Academia,Academia}               1
{Academia,Academia}                        1
Name: count, dtype: int64

In [ ]:
models